In [1]:

import rich
import rich.console
import transformers
import torch


In [102]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
m = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
t = transformers.AutoTokenizer.from_pretrained(model_name)

t.padding_side = "left"
t.pad_token = t.eos_token
input_tokens = t([
    "Q: AAA; A: 3; Q: AAAAAA; A: 6; Q: A, A: 1;",
], return_tensors="pt", padding=True).to(0)



forward_outputs = m(
    **input_tokens,
    return_dict=True,
)

forward_outputs.past_key_values = [
    [y[:, :, :-1] for y in x] 
    for x in forward_outputs.past_key_values
]

new_gen = m.generate(
    input_ids=input_tokens["input_ids"],
    past_key_values=forward_outputs.past_key_values,
    bos_token_id=t.eos_token_id, 
    pad_token_id=t.eos_token_id, 
    max_new_tokens=100, 
    use_cache=True,
)

for text in t.batch_decode(new_gen, skip_special_tokens=False):
    print("\n" + "#" * 80 + "\n\n" + text)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]